# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
csv_path = "output_data/cities.csv"
weather_df = pd.read_csv(csv_path)

weather_df = weather_df.loc[:, ~weather_df.columns.str.contains('^Unnamed')]
weather_df = weather_df.rename(columns={"Wind Speed":"Wind_Speed"})

weather_df.head()

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind_Speed
0,richards bay,90,US,1569909197,93,46.84,-95.91,46.40,12.75
1,cape town,0,CL,1569909197,100,-36.41,-71.90,39.20,4.70
2,haicheng,0,CN,1569909198,37,40.85,122.74,84.29,14.18
3,punta arenas,75,PH,1569909199,74,14.18,121.12,89.01,13.87
4,norman wells,1,US,1569909087,74,35.22,-97.44,79.00,16.11


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Configure gmaps
gmaps.configure(api_key=g_key)

#
fig = gmaps.figure()
locations = weather_df[["Lat", "Lng"]]
humidity = weather_df["Humidity"].astype(float)

#heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity,
                                dissipating=False, max_intensity=100,
                                point_radius=3)

fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
# set humidity conditions
humidityBelow80 = weather_df[weather_df.Humidity < 80]
humidityBelow80.head()

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind_Speed
2,haicheng,0,CN,1569909198,37,40.85,122.74,84.29,14.18
3,punta arenas,75,PH,1569909199,74,14.18,121.12,89.01,13.87
4,norman wells,1,US,1569909087,74,35.22,-97.44,79.00,16.11
5,vostok,12,RU,1569909200,59,46.45,135.83,70.97,5.46
6,bluff,25,AU,1569909200,22,-23.58,149.07,86.81,3.02


In [6]:
humidityAbove70 = humidityBelow80[humidityBelow80.Humidity > 70]
humidityAbove70.head()

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind_Speed
3,punta arenas,75,PH,1569909199,74,14.18,121.12,89.01,13.87
4,norman wells,1,US,1569909087,74,35.22,-97.44,79.00,16.11
10,jalu,0,LY,1569909201,71,29.03,21.55,71.33,11.16
19,novopavlovka,75,KG,1569909204,71,42.87,74.49,55.40,2.24
20,arraial do cabo,0,BR,1569909204,77,-22.84,-46.65,62.60,18.34


In [8]:
# Set wind speed condition
windBelow10 = humidityAbove70[humidityAbove70.Wind_Speed < 10]
windBelow10.head()

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind_Speed
19,novopavlovka,75,KG,1569909204,71,42.87,74.49,55.40,2.24
25,portland,90,US,1569909182,76,43.66,-70.25,55.00,4.70
30,abu zabad,75,JP,1569909209,78,34.51,131.49,84.00,3.36
34,khrebtovaya,96,RU,1569909210,77,56.71,104.25,36.77,9.84
37,hambantota,75,LK,1569909211,74,6.12,81.12,86.00,3.36


In [9]:
# Set cloudiness to zero
zeroCloudiness = windBelow10[windBelow10.Cloudiness == 0]
zeroCloudiness.head()

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind_Speed
109,nizhniy odes,0,RU,1569909233,77,45.39,39.24,66.99,6.71
224,lagoa,0,PT,1569909270,77,37.14,-8.45,66.20,4.70
301,nikolayevskaya,0,RU,1569909296,76,43.22,44.21,60.80,2.24
306,krasnoye,0,RU,1569909297,77,44.87,39.46,66.99,6.71
320,hajmasker,0,HU,1569909303,71,47.15,18.02,55.40,6.93


In [10]:
# Drop any rows will null values.
hotel_df = zeroCloudiness.dropna()
hotel_df.head()

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind_Speed
109,nizhniy odes,0,RU,1569909233,77,45.39,39.24,66.99,6.71
224,lagoa,0,PT,1569909270,77,37.14,-8.45,66.20,4.70
301,nikolayevskaya,0,RU,1569909296,76,43.22,44.21,60.80,2.24
306,krasnoye,0,RU,1569909297,77,44.87,39.46,66.99,6.71
320,hajmasker,0,HU,1569909303,71,47.15,18.02,55.40,6.93


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [11]:
hotel_df['Hotel Name'] = ""
hotel_df

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind_Speed,Hotel Name
109,nizhniy odes,0,RU,1569909233,77,45.39,39.24,66.99,6.71,
224,lagoa,0,PT,1569909270,77,37.14,-8.45,66.20,4.70,
301,nikolayevskaya,0,RU,1569909296,76,43.22,44.21,60.80,2.24,
306,krasnoye,0,RU,1569909297,77,44.87,39.46,66.99,6.71,
320,hajmasker,0,HU,1569909303,71,47.15,18.02,55.40,6.93,
373,ende,0,ID,1569909322,76,-8.84,121.65,79.25,5.37,
387,iwanai,0,JP,1569909327,72,42.97,140.51,66.83,7.27,
468,novoukrainskoye,0,RU,1569909356,71,45.38,40.53,63.00,8.48,
487,puertollano,0,ES,1569909363,76,38.69,-4.11,64.99,1.12,


In [ ]:

params = {"radius": 5000
          "types": "hotel"
          "api_keys": g_key}

In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in narrowed_city_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [ ]:
# Add marker layer ontop of heat map


# Display Map